## Classical SDP relaxation of OPF
As a first step, we present a simple implementation of the classical SDP formulation of the Optimnal Power Flow problem using CVXPY.

### Setup
We start by importing pandapower for the test cases;
CVXPY, an open source Python-embedded modeling language for convex optimization problems;
MOSEK, a large scale optimization software. 

In [1]:
import pandapower.networks as nw
import pandapower as pp
import numpy as np
import cvxpy as cp
from scipy.sparse import csr_matrix
import mosek

### Load Network Data
Now,let's make a function that loads the IEEE test cases from Pandapower.

In [2]:
# Select IEEE test case
def load_network(case):
    if case == 14:
        return nw.case14()
    elif case == 30:
        return nw.case30()
    elif case == 57:
        return nw.case57()
    elif case == 118:
        return nw.case118()
    else:
        raise ValueError("Unsupported test case")

In [ ]:
# Load the chosen IEEE test case
case_number = 14 # Change to 30, 57, or 118 for larger cases
net = load_network(case_number)

pp.runpp(net)  # Run power flow to initialize values
net

This pandapower network includes the following parameter tables:
   - bus (14 elements)
   - load (11 elements)
   - gen (4 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (15 elements)
   - trafo (5 elements)
   - poly_cost (5 elements)
   - bus_geodata (14 elements)
 and the following results tables:
   - res_bus (14 elements)
   - res_line (15 elements)
   - res_trafo (5 elements)
   - res_ext_grid (1 element)
   - res_load (11 elements)
   - res_shunt (1 element)
   - res_gen (4 elements)

In [6]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.043029,0.0,0.0,0.0
1,0,gen,0.0,20.0,0.250000,0.0,0.0,0.0
2,1,gen,0.0,40.0,0.010000,0.0,0.0,0.0
3,2,gen,0.0,40.0,0.010000,0.0,0.0,0.0
4,3,gen,0.0,40.0,0.010000,0.0,0.0,0.0


When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accessed after power flow using net._ppc
We will get the bus admittance matrix using exactly that and extract all the parameters we need.

In [7]:
#Gneration bus information
net.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,min_vm_pu
0,1,135.000,b,1.0,True,1.06,0.94
1,2,135.000,b,1.0,True,1.06,0.94
2,3,135.000,b,1.0,True,1.06,0.94
3,4,135.000,b,1.0,True,1.06,0.94
4,5,135.000,b,1.0,True,1.06,0.94
5,6,0.208,b,1.0,True,1.06,0.94
6,7,14.000,b,1.0,True,1.06,0.94
7,8,12.000,b,1.0,True,1.06,0.94
8,9,0.208,b,1.0,True,1.06,0.94
9,10,0.208,b,1.0,True,1.06,0.94


In [15]:

#make sure all units are consistent (convert everything to per unit)
Sbase = net.sn_mva # unit value for power MVA
Vbase = 135 # unit value for voltage kV
Zbase = Vbase**2 / Sbase # base impedance in Ohms
Ybase = 1 / Zbase # base admittance in S

# Now, we extract network parameters
Ybus = net._ppc["internal"]["Ybus"].todense() # Bus admittance matrix (Already in per unit. We need todense since the matrices are stored as sparcse matrices, but dense are easier to work with)
n = len(net.bus)  # Number of buses
# slack_bus = net.gen.bus.values[0]  # Index of the slack bus
branch = net.line  # Get each transmission line's specification for line constraints
#we need to add line flow constraints

# Extract generator information for generation constraints and convert to per unit
gen_buses = net.gen["bus"].values #index of the generators.
min_p_pu = net.gen["min_p_mw"].values/Sbase #minimal real power
max_p_pu = net.gen["max_p_mw"].values/Sbase
min_q_pu = net.gen["min_q_mvar"].values/Sbase #minimal reactive power
max_q_pu = net.gen["max_q_mvar"].values/Sbase

# Extract load buses for generation constraints
load_buses = net.load["bus"].values #index of the load buses.
#load active and reactive power for load
p_load = np.zeros(n)
q_load = np.zeros(n)
for i,row in net.load.iterrows():
    p_load[row['bus']] = -row['p_mw']/Sbase #negative sign because pandapower defines load as positive, but we need it as negative for consumption
    q_load[row['bus']] = -row['q_mvar']/Sbase

# Voltage limits
min_v_pu = net.bus["min_vm_pu"].values ** 2  # Squared for SDP
max_v_pu = net.bus["max_vm_pu"].values ** 2

# Extract generator cost coefficients from net.poly_cost
cost = net.poly_cost.drop([0])
c2 = cost["cp2_eur_per_mw2"].values/(Sbase**2)   # Quadratic cost coefficients in EUR/pu^2
c1 = cost["cp1_eur_per_mw"].values/Sbase   # Linear cost coefficients in EUR/pu
c0 = cost["cp0_eur"].values/Sbase          # Constant cost coefficients in EUR/pu

# Ensure the cost coefficients have the correct size
if len(c2) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
if len(c1) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
if len(c0) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")

# Make the cost matrix
C = np.zeros((n, n))
for i, gen_bus in enumerate(gen_buses):
    C[gen_bus, gen_bus] = c2[i]

# Print the cost matrix
np.set_printoptions(precision=10, suppress=True)
print(C)

[[0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.      ]
 [0.       0.000025 0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.000001 0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.000001 0.       0.
  0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.000001
  0.       0.       0.       0.       0.       0.  

### Define Variables and the objective function
The problem we are trying to solve in SDP OPF is
$$min \space \space trCW$$
subjected to $W \succcurlyeq 0$ and other linear constraints, which are

**Active Power Flow**
$$p_{min, j} \le tr \Phi_jW_B \le p_{max, j}$$
where $\Phi_j = \frac{1}{2}(Y^H_j + Y_j)$

**Reactive Power Flow**
$$q_{min, j} \le tr \Psi_jW_B \le q_{max, j}$$
where $\Psi_j = \frac{1}{2i}(Y^H_j + Y_j)$

**Voltage Magnitude Constraints**
$$v_{min, j} \le tr J_jW_B \le v_{max, j}$$
where J is the Hermitian matrix with a single 1 in the (j,j)th entry and zero everywhere else.

In [21]:
# Define an arbitrary n x n Hermitian W (Directly enforcing W= V V^+ will require X to be rank-1, which is too strict. We use the power balance, PSD and voltage constraints to guide this arbitrary X to X = VV^+)
W = cp.Variable((n, n), hermitian=True)

# Define phi and psi matrices for each bus as the constraints
Phi = [] # Hermitian part of Y_j
Psi = [] # Skewed Hermitian part of Y_j
J = [] # Hermitian matrix with a single 1 in the (j,j)th entry and zero everywhere else.
for j in range(n):
    Y_j = np.zeros_like(Ybus,dtype=complex) #Y_j is the admittance matrix with only the j-th row, everything else is zero
    Y_j[j,:] = Ybus[j,:]
    Y_j_H = np.conj(Y_j).T #Hermitian of Y_j
    
    J_j= np.zeros_like(Ybus,dtype=complex)
    J_j[j,j] = 1

    Phi_j = 1/2 * (Y_j_H + Y_j) 
    Psi_j = 1/(2j) * (Y_j_H - Y_j)

    Phi.append(Phi_j) 
    Psi.append(Psi_j)
    J.append(J_j)


# Objective Function, tr(CW), which should be real since cost is real
#Even though the trace of the product of two Hermitian matrices is theoretically real, numerical errors during computation can introduce small imaginary components.
# Explicitly using cp.real() ensures that the solver only considers the real part of the objective function.
objective_function = cp.real(cp.trace(C @ W)) 
objective = cp.Minimize(objective_function)

# objective_terms = [
#                    c1[i] * cp.real(cp.trace(Ybus[int(bus), :] @ W @ Ybus[:, int(bus)])) +
#                    c0[i] for i, bus in enumerate(gen_buses)]
# objective = cp.Minimize(cp.sum(objective_terms))

# Constraints
constraints = []
# Positive semidefinite constraint
constraints.append(W >> 0)


for j in range(n):

    #Voltage magnitude constraints
    constraints.append(min_v_pu[j]<=cp.real(cp.trace(J[j] @ W)))
    constraints.append(cp.real(cp.trace(J[j] @ W))<=max_v_pu[j])

    # Power balance equations (real and reactive)
    Pin_j = cp.trace(Phi[j]@W)
    Qin_j = cp.trace(Psi[j]@W)

    # If bus is a generator bus, enforce generation limits
    if j in gen_buses:
        idx = np.where(gen_buses == j)[0][0]  # Get generator index
        constraints.append(min_p_pu[idx] <= cp.real(Pin_j))
        constraints.append(cp.real(Pin_j) <= max_p_pu[idx])
        constraints.append(min_q_pu[idx] <= cp.real(Qin_j))
        constraints.append(cp.real(Qin_j) <= max_q_pu[idx])
    else:
        # For non-generator buses, enforce power balance constraints
        constraints.append(cp.real(Pin_j) == p_load[j])
        constraints.append(cp.real(Qin_j) == q_load[j])


### Solve using cvxpy's SDP solver

In [46]:
# Solve the SDP
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)

# Print results
print("Problem Status:", prob.status)
print("Optimal cost:", prob.value*Sbase)
# print("Optimal voltage matrix W:\n", W.value)
P_g_sdp = [cp.real(cp.trace(Phi[j] @ W)).value * Sbase for j in gen_buses]
print("Real power generation in MW (SDP OPF):", P_g_sdp)
Q_g_sdp = [cp.imag(cp.trace(Psi[j] @ W)).value * Sbase for j in gen_buses]
print("Reactive power generation in MW (SDP OPF):", Q_g_sdp)
V_sdp = [np.sqrt(cp.real(W[j, j]).value) for j in range(n)]
print("Voltage magnitudes in kV(SDP OPF):", V_sdp)

Problem Status: optimal
Optimal cost: 0.0025038887102964424
Real power generation in MW (SDP OPF): [7.073221621450432, 16.517154040925895, 73.2378084061474, 49.2154151230394]
Reactive power generation in MW (SDP OPF): [2.220446049250313e-14, 1.1102230246251565e-14, 0.0, 0.0]
Voltage magnitudes in kV(SDP OPF): [0.9419110850681176, 0.9401290366667133, 0.9727530401269604, 0.9421783187031414, 0.9412684578668734, 1.0030405309652901, 0.9661669499964524, 0.9952302143383635, 0.9578021265489536, 0.9552528475553712, 0.9727205667182525, 0.9812953000980303, 0.9737376066184861, 0.9430733108603201]


### Benchmark using Classical OPF

In [48]:
pp.runopp(net)

#Print results
opf_cost = net.res_cost
print("OPF Cost ($/hr):", opf_cost)
P_g_opf = net.res_gen["p_mw"].values
print("OPF Real power generation (OPF):", P_g_opf)
Q_g_opf = net.res_gen["q_mvar"].values
print("OPF Reactive power generation (OPF):", Q_g_opf)
V_opf = net.res_bus["vm_pu"].values
print("OPF Voltage magnitudes (OPF):", V_opf)

gen vm_pu > bus max_vm_pu for gens [2 3]. Setting bus limit for these gens.


OPF Cost ($/hr): 8081.526613828872
OPF Real power generation (OPF): [36.7191562618 28.7425182685  0.000757014   8.4947885717]
OPF Reactive power generation (OPF): [23.6850953116 24.1268763329 11.5453138247  8.2729697691]
OPF Voltage magnitudes (OPF): [1.06         1.04075293   1.0156248896 1.0144605403 1.0163623191
 1.0599986635 1.0463462221 1.0599989651 1.0436982913 1.0391358121
 1.0460084674 1.0448191624 1.0399476443 1.0238876673]
